## 6. Kubernetes Controller
- API, etcd, scheduler 등의 상호작용과 함께 Pod의 갯수를 보장
- Replication Controller, Replicaset
- Deployment
- StateFul sets
- DaemonSets
- Job, CronJob 등이 존재

![kubenetes_controller_type](img/kubenetes_controller_type.png)

<hr>

### 6.1 Replication Controller (RC)
- 요구하는 파드의 갯수를 보장하며(Pod를 여러개 복제), 안정적으로 유지
- Replica수, Pod Selector, Pod Template 3개로 구성
- 필드
    - replicas: <배포갯수>
    - selector: 이하 (key:value) 
    - template: <템플릿>

- selector의 key 이름을 가진 파드를 찾아서, replicas 갯수만큼 운영해주고, 부족하면 template 형식으로 생성.
- kubectl 명령어
    - kubectl get replicationcontrollers 
    - kubectl get-rc
    - kubectl describe rc rg-nginx
    - kubectl scale rc rc-nging --replicas=2 : 갯수 조정 가능
- Rolling Back, Rolling update 지원 x
    - 변경 사항 (yaml)이 기록됨.
    - 자동으로 변경되지 않음, kubectl delete 해야 controller가 자동생성되면서, 수정버전 만듬
    - 등호기반 (레이블 선택시 key에 대한 value가 같은지)으로 접근

![kubernetes_controller_replication_controller](img/kubernetes_controller_replication_controller.png)

<hr>

### 6.2 ReplicaSet
- Replication Controller와 같은 역할을 하는 컨트롤러, 과거 버전에 쓰였던 것이며, ReplicaSet이 좀더 업그레이드 버전
- ReplicaSet 은 사실 포드와 느슨한 연결을 가짐. (실제 포드와 연결되있는 것이 아님)
    - 라벨 셀렉터를 이용한 느슨한 연결을 이룸
    - 크게 spec ~ selector : 라벨 셀렉터를 지칭 / template ~ : 포드 관련 정의
- 라벨은 포드 등의 쿠버네틱스 리소스를 분류시 유용하게 사용되는 메타데이터로 matchLabel을 통해 생성해야 되는 포드를 찾음.
- Replication Controller는 Equality(등호) 기반 Selector면 Replica Set은 집합(set) 기반의 Selector 이용 (보다 풍부한 Selector)
- matchExpressions 연산자 {key:value} 형태
    - ex) {key: abcd, operator: In, value:[“2.1”, “2.2”] }
        - key가 abcd, value가 2.1이거나 2.2인 Pod에 연결
    - ex) {key: abcd, operator: NotIn, value:[“2.1”, “2.2”] }
        - key가 version이고, value는 2.1, 2.2가 아닌 Pod에 연결
    - ex) {key: abcd, operator: Exists }
        - key와 맞는 label의 pod를 연결
    - ex) {key: abcd, operator: DoseNotExist }
        - key와 다른 label의 pod를 연결

- kubectl 명령어
    - kubectl get replicasets

![kubenetes_controller_replicaset](img/kubenetes_controller_replicaset.png)

<hr> 

### 6.3 Deployment
- Replication Controller와 Replica Set의 가장 일반적으로 쓰이는 최상위 추상화 개념
- 컨테이너 애플리케이션 배포 및 관리하는 역할 담당
    - 애플리케이션 업데이트시 변경 사항 저장하는 revision을 남겨 롤백 지원
    - 무중단 서비스를 위해 포드의 롤링 업데이터 전략 지정, 다양한 정책 사용 가능
- ReplicaSet을 컨트롤해서 Pod수를 조절 가능
- Rolling Update, Rolling Back 지원 O
    - ex) 서비스 운영중인 Nginx 1.14를 1.15로 바꾸는 것
    - 고객의 UI에 영향을 미치지 않는 선에서 지원해서 서비스의 지속성을 유지
    
![kubenetes_controller_deployment](img/kubenetes_controller_deployment.png)

- Rolling Update
    - kubectl set image deployment <deploy_name> <container_name> = <new_version_image>
    - kubectl set image deployment app-deploy app=nginx:1.15 --record
    - 이름 app-deploy인 deployment의 nginx:1.15를 기록하면서 교체 

- RollBack
    - kubectl rollout history deployment <deploy_name>
        - 이전의 update 기록을 보여줘.
    - kubectl rollout undo deploy <deploy_name>
    
![kubenetes_controller_deployment_rollback](img/kubenetes_controller_deployment_rollback.png)

- 예제 순서
    - 1) kubectl create -f <file_name>.yaml --record
    - 2) kubectl set image deployment app-deploy web=nginx:1.15 --record
    - 3) kubectl rollout pause deployment app-deploy : 동작 일시정지 상태확인
    - 4) kubectl rollout resume deployment app-deploy : 동작 다시 실행
    - 5) kubectl rollout history deployment app-deploy : history 확인
    - 6) kubectl rollout undo deployment app-deploy : 바로 전단계로 돌아가기
    - 7) kubectl rollout undo deployment app-deploy --to-revision=3 : 3단계 이전
    - 8) yaml 파일로 Rolling update하기
        - yaml 파일에 annotations : kubernetes.io/change-cause:version 1.14 작성 (기본 1.14)
        - kubectl create -f file-name.yaml
        - yaml 파일 1.15로 수정
        - kubectl apply -f file-name.yaml을 하면은 자동으로 1.15로 Rolling update
![kubenetes_controller_deployment_yaml](img/kubenetes_controller_deployment_yaml.PNG)
    
- 필드
    - revisionHistoryLimit : 10 
        - history 최대 몇개 저장
    - strategy: rollingUpdate: maxSurge : 25%, maxUnavailable: 25%
        - container에 대한 운영을 해서 얼마나 빠르게 업데이트 할껀지)
<hr>

### (4) Daemonset
- 로그나 모니터링 등 클러스터 내의 모든 노드에 설치가 필요한 경우 사용
- 새로운 노드 추가 나 노드 한개 삭제시 자동으로 파드 실행 및 삭제
- Rolling update, back 기능도 가지고 있음
- 편집기(yaml) 수정 후 apply 시 반영됨

<hr>

### 6.5 Statefulset
- 기존 컨트롤러와 달리 상태가 있는 파드 들을 관리하는 컨트롤러
- 실행 순서를 보장하며 순서나 데이터가 중요한 경우에 사용
- 여러 개의 파드 사이에 순서를 지정해서 실행 가능 
    - N개의 레플리카로 설정하면 차례대로 {0...N} 순으로 파드 생성
    - 레플리카 갯수 늘리거나 줄일시(스케일링) 순차 파드 생성 및 종료
    - 만들어지거나 종료될 때 이전 파드를 기다림
- 파드 관리 정책
    - sepc.podManagementPolicy : OrderReady 관리
        - 기본 필드값으로 파드를 순서대로 관리
    - sepc.podManagementPolicy : Parallel 관리
        - 파드들이 순서 없이 병렬로 실행되거나 종료가능
        - 이전 파드를 기다리지 않고 바로 실행

<hr>

### 6.6 Job
- Job은 실행 된 후 종료해야 하는 성격의 작업을 실행 시 사용하는 컨트롤러
- Batch 처리하는 Pod는 작업이 완료되면 종료됨
- Batch 처리에 적합한 컨트롤러로 Pod의 성공적인 완료를 보장
    - 비정상 종료시 다시 실행, 정상 종료 시 완료
    - 단 Job이 성공적으로 정상 종료되도 파드 자체를 삭제하지는 않음. 
- 에러나 로그에서 에러나 경고 등 확인 가능.

### 6.7 Cron Job (크론잡)
- Job을 시간 기준으로 관리
    - ex) 지정한 시간에 한번만 Job을 실행 및 지정한 시간동안 주기적으로 Job 반복수행 등

